In [56]:
from env import host, user, password
from env import get_db_url
import pandas as pd
import numpy as np
from pydataset import data
url = f'mysql+pymysql://{user}:{password}@{host}/employees'


In [57]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [58]:
#this is done in env.py and imported above

In [59]:
test_url = get_db_url('employees')

In [60]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', test_url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


a. Intentionally make a typo in the database url. What kind of error message do you see?

In [61]:
test_url = get_db_url('employeeasdass') # employees spelled wrong intentionally
# pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', test_url)

# OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'innis_1662'@'%' to database 'employeeasdass'")
# (Background on this error at: https://sqlalche.me/e/14/e3q8)

b. Intentionally make an error in your SQL query. What does the error message look like?

In [62]:
test_url = get_db_url('employees')
#pd.read_sql('SELECTTTTTTT * FROM employees LIMIT 5 OFFSET 50', test_url) # select intentionally wrong
# error is ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'SELECTTTTTTT * FROM employees LIMIT 5 OFFSET 50' at line 1")
# [SQL: SELECTTTTTTT * FROM employees LIMIT 5 OFFSET 50]
# (Background on this error at: https://sqlalche.me/e/14/f405)

Read the employees and titles tables into two separate DataFrames.



In [63]:
employees = pd.read_sql('SELECT * FROM employees', url)
titles = pd.read_sql('SELECT * FROM titles', url)

How many rows and columns do you have in each DataFrame? Is that what you expected?



In [64]:
employees.shape

(300024, 6)

In [65]:
titles.shape

(443308, 4)

Display the summary statistics for each DataFrame.




In [66]:
employees.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [67]:
titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


How many unique titles are in the titles DataFrame?



In [68]:
titles.value_counts('title').count()


7

In [69]:
titles['title'].unique().size

7

What is the oldest date in the to_date column?



In [70]:
titles.sort_values(by='to_date')['to_date'].head(1)



16064    1985-03-01
Name: to_date, dtype: object

What is the most recent date in the to_date column?



In [71]:
new_titles = pd.read_sql('SELECT * FROM titles WHERE to_date != "9999-01-01"', url)
new_titles.sort_values(by='to_date')['to_date'].tail(1)

167151    2002-08-01
Name: to_date, dtype: object

Exercises II

Copy the users and roles DataFrames from the examples above.


In [72]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [73]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter



What is the result of using a right join on the DataFrames?
 


In [74]:
# Perform an right join.

users.merge(roles, left_on='role_id', right_on='id', how='right')
# Perform an outer join specifying the left and right DataFrame keys.




,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


What is the result of using an outer join on the DataFrames?


In [75]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)


,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only



What happens if you drop the foreign keys from the DataFrames and try to merge them?


In [76]:
users.merge(roles, how='outer', indicator=True)


,id,name,role_id,_merge
0,1,bob,1.0,left_only
1,2,joe,2.0,left_only
2,3,sally,3.0,left_only
3,4,adam,3.0,left_only
4,5,jane,NaN,left_only
5,6,mike,NaN,left_only
6,1,admin,NaN,right_only
7,2,author,NaN,right_only
8,3,reviewer,NaN,right_only
9,4,commenter,NaN,right_only



Load the mpg dataset from PyDataset.


In [77]:
#data('mpg', show_doc=True) # view the documentation for the dataset
mpg = data('mpg') # load the dataset and store it in a variable

mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact



Output and read the documentation for the mpg dataset.


In [78]:
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 





How many rows and columns are in the dataset?


In [79]:
mpg.shape

(234, 11)


Check out your column names and perform any cleanup you may want on them.


In [80]:
mpg.columns

Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [81]:
mpg = mpg.rename(columns={'fl':'fuel_type'}) 


Display the summary statistics for the dataset.


In [82]:
mpg.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


In [83]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fuel_type     234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB



How many different manufacturers are there?


In [84]:
mpg.manufacturer.value_counts().count()

15


How many different models are there?

In [85]:
mpg.model.value_counts().count()

38


Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.


In [86]:
mpg['mileage_difference'] = mpg.hwy - mpg.cty
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fuel_type,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8



Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.


In [87]:
mpg['average_mileage'] = (mpg.hwy + mpg.cty)/2
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fuel_type,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0



Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.
 

In [88]:
mpg['is_automatic'] = mpg.trans.str.contains('auto')
mpg.sample(5)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fuel_type,class,mileage_difference,average_mileage,is_automatic
18,audi,a6 quattro,4.2,2008,8,auto(s6),4,16,23,p,midsize,7,19.5,True
23,chevrolet,c1500 suburban 2wd,6.0,2008,8,auto(l4),r,12,17,r,suv,5,14.5,True
12,audi,a4 quattro,2.8,1999,6,auto(l5),4,15,25,p,compact,10,20.0,True
219,volkswagen,jetta,2.5,2008,5,manual(m5),f,21,29,r,compact,8,25.0,False
117,hyundai,tiburon,2.0,1999,4,manual(m5),f,19,29,r,subcompact,10,24.0,False



Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?


In [89]:
mpg.groupby('manufacturer').average_mileage.mean().nlargest(n=1,keep ='all')     
#df.groupby('classroom').math.max()
   
   

manufacturer
honda    28.5
Name: average_mileage, dtype: float64


Do automatic or manual cars have better miles per gallon?

In [90]:
mpg.groupby('is_automatic').average_mileage.mean()
#manual is better

is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

Exercises III
Use your get_db_url function to help you explore the data from the chipotle database.


In [91]:
chipotle_url = get_db_url('chipotle')
chipotle = pd.read_sql('SELECT * FROM orders', chipotle_url)

In [92]:
chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98



What is the total price for each order?


In [93]:
def launder_the_cash(money):
    return float(money.replace('$','').replace(',',''))

In [94]:
#convert '$item price'  string to usuable float
chipotle.item_price = chipotle.item_price.apply(launder_the_cash)

In [95]:
chipotle.groupby('order_id').item_price.sum()

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64


What are the most popular 3 items?


In [96]:
chipotle.item_name.value_counts().head(3)

Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64


Which item has produced the most revenue?


In [97]:
chipotle.groupby('item_name').item_price.sum().nlargest(n=1, keep='all')

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64


Join the employees and titles DataFrames together.


In [98]:
employees_url= get_db_url('employees')
employees = pd.read_sql('SELECT * FROM employees', employees_url)
titles = pd.read_sql('SELECT * FROM titles', employees_url)

In [99]:
employees_with_title= employees.merge(titles, left_on='emp_no', right_on='emp_no', how='inner', indicator=True)
employees_with_title.head()


,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date,_merge
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,both
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01,both
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01,both
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01,both
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01,both



For each title, find the hire date of the employee that was hired most recently with that title.


In [100]:
most_recent_title= pd.DataFrame(employees_with_title.groupby('title').hire_date.max())
most_recent_title= most_recent_title.merge(employees_with_title, left_on=['title','hire_date'], right_on=['title','hire_date'], how='inner')
most_recent_title[['title','hire_date','first_name','last_name']]

,title,hire_date,first_name,last_name
0,Assistant Engineer,1999-12-24,Munehiro,Luke
1,Engineer,2000-01-28,Bikash,Covnot
2,Manager,1992-02-05,Oscar,Ghazalie
3,Senior Engineer,2000-01-01,Mariangiola,Boreale
4,Senior Staff,2000-01-13,Volkmar,Perko
5,Staff,2000-01-12,Ulf,Flexer
6,Technique Leader,1999-12-31,Karlis,Orsini



Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [105]:
#bring in df with current data (to_date > now) and department names
current_departments = pd.read_sql('SELECT * FROM dept_emp WHERE to_date = "9999-01-01"', employees_url)
current_titles = pd.read_sql('SELECT * FROM titles WHERE to_date = "9999-01-01"', employees_url)
department_names = pd.read_sql('SELECT * FROM departments', employees_url)

In [106]:
#add dept name to current dept
current_departments = current_departments.merge(department_names, left_on='dept_no', right_on='dept_no', how='inner')

In [107]:
#drop the non needed columns
current_departments.drop(columns=['from_date','to_date','dept_no'],inplace=True)
current_titles.drop(columns=['from_date','to_date'],inplace=True)


In [108]:
current_departments

,emp_no,dept_name
0,10001,Development
1,10006,Development
2,10012,Development
3,10014,Development
4,10022,Development
...,...,...
240119,499771,Finance
240120,499868,Finance
240121,499873,Finance
240122,499977,Finance


In [110]:
titles_and_dept = current_titles.merge(current_departments, left_on='emp_no', right_on='emp_no', how='inner')
pd.crosstab(titles_and_dept.title, titles_and_dept.dept_name)


dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,68,1652,0,0,0,1402,389,77,0
Engineer,627,14040,0,0,0,12081,3405,830,0
Manager,1,1,1,1,1,1,1,1,1
Senior Engineer,1790,38816,0,0,0,33625,9458,2250,0
Senior Staff,11268,1085,9545,9824,11290,1123,0,9092,28797
Staff,3574,315,2891,3073,3551,349,0,2870,8903
Technique Leader,241,5477,0,0,0,4723,1293,321,0
